In [21]:
using MAT
include("ELM.jl")

In [ ]:
function build_X_T(dset::Dict, input_vars::Array{String,1}, output_vars::Array{String,1}; clusters_no::Integer = 1, train_ratio::Real = 0.7, output_transform::Function = x -> x, k_ahead::Integer = 0)
    X_train = Vector{Matrix{Real}}(undef, 0);
    T_train = Vector{Matrix{Real}}(undef, 0);
    X_test = Vector{Matrix{Real}}(undef, 0);
    T_test = Vector{Matrix{Real}}(undef, 0);

    b_dt = [137, -300, 300, -200, 75, -12];
    a_dt = [(60*StatsBase.mode(dset["time"][1][2:end]-dset["time"][1][1:end-1]))];

    kmeans_matr = Matrix{Real}(undef, 0, 5);

    for exp_output ∈ map(output_transform, dset[output_vars[1]])
        kmeans_matr = vcat(kmeans_matr,
            [Statistics.mean(exp_output),
                Statitstics.std(exp_output),
                maximum(exp_output),
                minimum(exp_output),
                Statistics.mean(DSP.filt(b_dt, a_dt, exp_output).^2)]');
    end

    norm_metrics = kmeans_matr .- minimum(kmeans_matr, dims=1);
    norm_metrics ./= maximum(norm_metrics, dims=1);
    exp_classes = Clustering.kmeans(norm_metrics', clusters_no).assignments

    function build_experiment(exp_idx::Integer)
        exp_input = Matrix{Real}(undef, length(dset["time"][exp_idx]), 0);
        exp_output = Matrix{Real}(undef, length(dset["time"][exp_idx]), 0);

        for input_var ∈ input_vars
            exp_input = hcat(exp_input, dset[input_var][exp_idx]);
        end
        for output_var ∈ output_vars
            exp_output = hcat(exp_output, dset[output_var][exp_idx]);
        end

        return exp_input[1:end-k_ahead,:], exp_output[k_ahead+1:end,:]
    end

    for class_no ∈ range(1, stop=clusters_no)
        indices = Random.shuffle(findall(x -> x==class_no, exp_classes));
        train_len = floor(Integer, train_ratio*length(indices));

        train_idxs = indices[1:train_len];
        test_idxs = indices[train_len+1:end];

        for exp_idx ∈ train_idxs
            exp_input, exp_output = build_experiment(exp_idx);
            push!(X_train, exp_input);
            push!(T_train, exp_output);
        end

        for exp_idx ∈ test_idxs
            exp_input, exp_output = build_experiment(exp_idx);
            push!(X_test, exp_input);
            push!(T_test, exp_output);
        end
    end

    return X_train, T_train, X_test, T_test;
end

In [22]:
file = matopen("dataset.mat")
dset = read(file, "dset")
close(file)

size(dset["Lambda_TB"][1],1)

2415

In [23]:
X_train, T_train, X_test, T_test = build_X_T(dset, ["Speed", "Throttle_MsrdPstn", "Lambda_TB"], ["Lambda_TB"], clusters_no = 5);

In [26]:
a = [1, 1, 1]
b = 1
data = rand([0,1], 10, 3)

# test_neuron = RBF_neuron(a, b, gaussian)
typeof(gaussian)

typeof(gaussian)